## Load Dataset

In [1]:
import pickle

In [2]:
def load_obj(name ):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
d = load_obj("council_minutes_dict")

## Pre-processing

In [4]:
import re

In [5]:
def clean_text(text):
    text = text.decode("UTF-8")
    text = text.replace('\n'," ")
    text = text.replace('\x0c'," ")
    text = re.sub(r"-", " ", text) # Split the words with "-" (for example：pre-processing ==> pre processing）
    text = re.sub(r"\d+/\d+/\d+", "", text) # Take out the dates
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) # Take out the time
    text = re.sub(r"[\w]+@[\.\w]+", "", text) # Take out the emails
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text) # Take out the websites
    pure_text = ''
    # Validate to check if there are any non-text content 
    for letter in text:
        # Keep only letters and spaces
        if letter.isalpha() or letter==' ':
            pure_text += letter
    # Join the words are not stand-alone letters
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    return text

In [6]:
# stop_words = ENGLISH_STOP_WORDS.union(word for word in ['docx','fyi','fw','get','see','ok','pm','whose','would','pls','thx','yes','print','okay','pis'])

In [7]:
text = clean_text(d['10'])

## Lemmatization

In [8]:
import spacy
import string
# Create our list of punctuation marks
punctuations = string.punctuation
# Load English tokenizer, tagger, parser, NER and word vectors
parser = spacy.load('en_core_web_sm')
# Create our list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return ' '.join(mytokens)

In [9]:
text = spacy_tokenizer(text)

## Steminization

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer

def nltk_stemmer(text):
    porter_stemmer = PorterStemmer()
    word_data = text
    # First Word tokenization
    nltk_tokens = nltk.word_tokenize(word_data)
    #Next find the roots of the word
    t = []
    for w in nltk_tokens:       
        t.append(porter_stemmer.stem(w))
    return t

In [11]:
text = nltk_stemmer(text)

## TF-IDF

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

In [13]:
# text = [text]

In [14]:
tfidf_vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)

word_matrix = tfidf_vectorizer.fit_transform(text)

vocab = tfidf_vectorizer.get_feature_names()

## LDA

In [16]:
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [17]:
lda_tfidf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf.fit(word_matrix)

C:\Users\Silver\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=5, random_state=0)

In [18]:
pyLDAvis.sklearn.prepare(lda_tfidf, word_matrix, tfidf_vectorizer)

C:\Users\Silver\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.297840  0.114266       1        1  21.171613
4     -0.191936  0.247915       2        1  20.702367
1     -0.033426 -0.103694       3        1  19.673004
2     -0.043759 -0.176950       4        1  19.521690
0     -0.028719 -0.081537       5        1  18.931326, topic_info=        Term       Freq      Total Category  logprob  loglift
109     citi  87.000000  87.000000  Default  30.0000  30.0000
574      san  46.000000  46.000000  Default  29.0000  29.0000
342     item  40.000000  40.000000  Default  28.0000  28.0000
505  project  37.000000  37.000000  Default  27.0000  27.0000
42    approv  37.000000  37.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
393    major   5.309193   5.964422   Topic5  -4.7688   1.5480
280    great   5.309193   5.964422   Topic5  -4.7688   1.5480
166       cp   4.452852   5.108083   Topic5  -4.9447   1.5271
238    exist   4.452852   5.108083   Topic5  -4.9447   1.5271
48   asphalt   4.452852   5.108083   Topic5  -4.9447   1.5271

[185 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         5  0.879667         aa
4         2  0.928111     accept
8         2  0.926454        act
9         2  0.990575     action
14        5  0.911803  administr
...     ...       ...        ...
674       2  0.927374     unanim
683       2  0.881962        use
693       3  0.900031      water
699       3  0.900371       year
700       5  0.958489       zone

[155 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 3, 1])